# Create Main Meter Only
Subsets the data to only main meters.  This eliminates the dual impact of sub-meters.

In [0]:
%run ../Utilities/ConfigUtilities

In [0]:
# Set up the environment using a function in ConfigUtilties.
set_spark_config()

In [0]:
from pyspark.sql.functions import col

uri = CONTAINER_URI_PATH
meter_path = f"{uri}/Bronze/iVUE/DIM_MeterInfo/DIM_MeterInfo.csv"
debug = 1

In [0]:
# Get all the main meters
meter_info_df =  spark.read.option("header", True).csv(meter_path)
meter_info_df = meter_info_df.select(['BI_ACCT', 'BI_MTR_NBR', 'BI_MTR_POS_NBR', 'BI_MTR_CONFIG'])

if debug:
    display(meter_info_df)
    print(meter_info_df.count())

In [0]:
main_meter_df = meter_info_df.filter(meter_info_df.BI_MTR_CONFIG.isNull())

if debug:
    display(main_meter_df)
    print(main_meter_df.count())



In [0]:
# Read the meter data from the silver table.
meter_data_df = spark.read.table(MDM_INDEXED_TABLE)

In [0]:
# Join the main meters with the meter data.
main_meter_df = main_meter_df.select('BI_MTR_NBR')
main_meter_df = main_meter_df.withColumnRenamed('BI_MTR_NBR', 'MeterNumber')
main_meter_data_df = meter_data_df.join(main_meter_df, on='MeterNumber', how='inner')

if debug:
    display(main_meter_data_df)

In [0]:
# Save to SilverEnhanced section.
main_meter_data_df.write.format("delta") \
            .mode("overwrite") \
            .save(MDM_MAIN_METER_ONLY_DATA_PATH)


In [0]:
# Vacuum
delta_table = DeltaTable.forPath(spark, MDM_MAIN_METER_ONLY_DATA)
delta_table.vacuum()  